In [ ]:
from mlflow.tracking import MlflowClient

In [ ]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [ ]:
# create a new experiment
client.create_experiment(name="my_138_experiment") # returns the experiment ID

In [ ]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='3', # mlops_nyc_taxi
    filter_string="metrics.rmse < 10", # grab only the runs with an RMSE below 10
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"] # Ascending
)

In [ ]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}") # round to 4 d.p

In [9]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)


In [11]:
# register the top performing model(run) from the mlops_nyc_taxi experiment
run_id = "cf138656ed814637ac616a10a9d3b17b" # 
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc_taxi_xgb")

Registered model 'nyc_taxi_xgb' already exists. Creating a new version of this model...
2023/05/29 00:24:29 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc_taxi_xgb, version 2
Created version '2' of model 'nyc_taxi_xgb'.


<ModelVersion: aliases=[], creation_timestamp=1685309069613, current_stage='None', description=None, last_updated_timestamp=1685309069613, name='nyc_taxi_xgb', run_id='cf138656ed814637ac616a10a9d3b17b', run_link=None, source='/Users/yahyamohnd/Desktop/DataTalkClub/MLOPS/week2_exp_mlops/mlruns/2/cf138656ed814637ac616a10a9d3b17b/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [12]:
model_name = "nyc_taxi_xgb"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: None


In [13]:
model_version = 2
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False # so version 1 will remain in Staging
)

<ModelVersion: aliases=[], creation_timestamp=1685309069613, current_stage='Staging', description=None, last_updated_timestamp=1685309129520, name='nyc_taxi_xgb', run_id='cf138656ed814637ac616a10a9d3b17b', run_link=None, source='/Users/yahyamohnd/Desktop/DataTalkClub/MLOPS/week2_exp_mlops/mlruns/2/cf138656ed814637ac616a10a9d3b17b/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [14]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1685309069613, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2023-05-29', last_updated_timestamp=1685309164444, name='nyc_taxi_xgb', run_id='cf138656ed814637ac616a10a9d3b17b', run_link=None, source='/Users/yahyamohnd/Desktop/DataTalkClub/MLOPS/week2_exp_mlops/mlruns/2/cf138656ed814637ac616a10a9d3b17b/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [15]:
model_version = 1
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False # 
)

from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1685308309697, current_stage='Production', description='The model version 1 was transitioned to Production on 2023-05-29', last_updated_timestamp=1685309197442, name='nyc_taxi_xgb', run_id='cf138656ed814637ac616a10a9d3b17b', run_link='', source='/Users/yahyamohnd/Desktop/DataTalkClub/MLOPS/week2_exp_mlops/mlruns/2/cf138656ed814637ac616a10a9d3b17b/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [16]:
# import packages
from sklearn.metrics import mean_squared_error
import pandas as pd

In [17]:
# create our functions

def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts) # not fitting the pre-processor again


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [19]:
df = read_dataframe("../yellow_tripdata_2022-03.parquet")

In [21]:
import pickle

with open("models/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [22]:
X_test = preprocess(df, dv)

In [23]:
target = "duration"
y_test = df[target].values

In [24]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test) # magic command % to clock the test

2023/05/29 00:35:59 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[00:35:59] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 8.21 s, sys: 272 ms, total: 8.48 s
Wall time: 1.41 s


{'rmse': 6.505082976506096}